In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

import os, glob
from os.path import join as pjoin
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean
from scipy import stats
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

---------------

In [3]:
import tifffile as tiff
import rasterio
from rasterio.windows import Window

In [4]:
data_path = "./data/train_tier_1/"
acc_data_path = pjoin(data_path, "acc")
acc_out_path = pjoin(os.getcwd(), "out")
if not os.path.exists(acc_out_path):
    os.mkdir(acc_out_path)
scene_id = "665946"
scene_dir_path = pjoin(acc_data_path, scene_id)
scene_labels_path = pjoin(acc_data_path, scene_id+"_labels")
scene_path = pjoin(scene_dir_path, scene_id+".tif")

In [69]:
# Read the input image
input_source = rasterio.open(scene_path)

print(input_source.width, input_source.height)

tiff_size = [input_source.profile.get("width"),
             input_source.profile.get("height")]

# List of overviews from biggest to smallest
oviews = input_source.overviews(1)

# Retrieve the smallest thumbnail
print(oviews)
oview = oviews[-2]
print('Decimation factor= {}'.format(oview))

bands = (1, 2, 3, 4)
# Tile size
tile_size = (len(bands), int(input_source.height//oview), int(input_source.width//oview))

img = np.empty(shape=tile_size).astype(input_source.profile['dtype'])
# tile_window = Window(1000*oview, 1000*oview, tile_size[1], tile_size[2])
img = input_source.read(bands, 
                         out=img)#, window=tile_window)

# Convert the values into float
img = img.astype('float')

img = img/np.max(img)
img = img.astype(np.float16)
img_out_path = pjoin(acc_out_path, scene_id+".png")

tiff.imsave(img_out_path, data=img[:3,:,:])

84466 150147
[2, 4, 8, 16, 32]
Decimation factor= 16


In [5]:
# Read the input image
input_source = rasterio.open(scene_path)

print(input_source.width, input_source.height)

tiff_size = [input_source.profile.get("width"),
             input_source.profile.get("height")]

# List of overviews from biggest to smallest
oviews = input_source.overviews(1)

# Retrieve the smallest thumbnail
oview = oviews[-1]
print('Decimation factor= {}'.format(oview))

bands = (1, 2, 3, 4)
# Tile size
tile_size = (len(bands), 1024, 1024)

xx, yy = np.meshgrid(np.arange(0, tiff_size[0], tile_size[1]),
                     np.arange(0, tiff_size[1], tile_size[2]),
                     sparse=True)
for x in xx.ravel():
    for y in yy.ravel():
        x = int(min(x, tiff_size[0]))
        y = int(min(y, tiff_size[1]))

        img = np.empty(shape=tile_size).astype(input_source.profile['dtype'])
        tile_window = Window(x, y, tile_size[1], tile_size[2])
        img = input_source.read(bands, 
                                 out=img, window=tile_window)

        # Convert the values into float
        img = img.astype('float')

        if np.max(img) <= 0:
            continue

        if np.count_nonzero(img[3,:,:].ravel())/(tile_size[1]*tile_size[2]) <= 0.25:
            continue

        img = img/np.max(img)
        img = img.astype(np.float16)
        img_out_path = pjoin(acc_out_path, scene_id+"_"+str(x)+"_"+str(y)+".png")
        tiff.imsave(img_out_path, data=img[:3,:,:])

84466 150147
Decimation factor= 32


KeyboardInterrupt: 